# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [33]:
from selenium import webdriver #importando o webdriver da bib selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By



    
 #abrir navegador
navegador = webdriver.Chrome()
navegador.get('https://www.google.com/')

#encontrando o campo de busca no google para pesquisar a cotação do dólar 
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#Coleta  da informação
cotacao_dolar = navegador.find_element(By.XPATH, 
                                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)

#encontrando o campo de busca no google para pesquisar a cotação do euro
navegador.get('https://www.google.com/')
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

##encontrando  a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)
navegador.quit()

    


5.0961
5.527484865
315.12


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [34]:
#Importando a lista de produtos
import pandas as pd
tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [35]:
#Atualizar a coluna cotação

#Localizando as linhas com DÓLAR, EURO E OURO
#tabela[coluna] -- coluna da cotação será alterada, recebendo a info atulaizada -> float(cotacao_dolar)
#tabela.loc[linha,coluna] = float()
#linhas => todas as linhas da coluna "Moeda" serão alteradas se forem = "Dólar" --->>> tabela["Moeda"] == "Dólar"

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizando o preço base de compra em reais (preço original * cotação)

tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizando o preço de venda (preço de compra * margem)

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]


display(tabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.096100,5096.049039,1.40,7134.468655
1,Carro Renault,4500.00,Euro,5.527485,24873.681893,2.00,49747.363785
2,Notebook Dell,899.99,Dólar,5.096100,4586.439039,1.70,7796.946366
3,IPhone,799.00,Dólar,5.096100,4071.783900,1.70,6922.032630
4,Carro Fiat,3000.00,Euro,5.527485,16582.454595,1.90,31506.663730
5,Celular Xiaomi,480.48,Dólar,5.096100,2448.574128,2.00,4897.148256
6,Joia 20g,20.00,Ouro,315.120000,6302.400000,1.15,7247.760000


### Agora vamos exportar a nova base de preços atualizada

In [36]:
tabela.to_excel("Produtos-Atualizado.xlsx", index = False)

# ENVIANDO POR EMAIL BASE ATUALIZADA COM PYAUTOGUI 
   (email já logado)

In [55]:
import pyautogui as pa
import pyperclip as pc
import time

pa.hotkey('ctrl', 't')
time.sleep(5)
pc.copy('https://mail.google.com/mail/u/0/#inbox?compose=new')
pa.hotkey('ctrl','v')
pa.press('enter')
time.sleep(25)

#preechendo campo 'para'
pc.copy('raiane.ssantos@outlook.com')
pa.hotkey('ctrl','v')
pa.press('tab')
#preechendo campo 'assunto'
pc.copy('Base de dados de produtos atualizada')
pa.hotkey('ctrl','v')
pa.press('tab')

#Preechendo corpo do e-mail, anexando o arquivo excel
pa.click(x=966, y=697)
time.sleep(5)
pa.click(x=646, y=48)

pc.copy('Downloads')  #digitando o caminho onde está o arquivo na barra
pa.hotkey('ctrl','v') 
pa.press('enter')
time.sleep(5)
pa.click(x=774, y=667)
pc.copy(r'Produtos-Atualizado.xlsx')  #digitando o nome do arquivo
pa.hotkey('ctrl','v')
time.sleep(3)
pa.press('enter')

#Envio do e-mail por meio de atalho 
pa.hotkey('ctrl','enter')




















Defaulting to user installation because normal site-packages is not writeable


Point(x=774, y=667)